<a href="https://colab.research.google.com/github/kristynpantoja/bsg_pytorch/blob/master/BSGcollab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math


from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import argparse
from types import SimpleNamespace

# import gensim.downloader as api
# from gensim.models import Word2Vec, FastText, KeyedVectors
# from os.path import isfile

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [25]:
# arguments 



window = 2




In [4]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kristyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kristyn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# # preprocessing stuff

# def remove_non_ascii(words):
#     """Remove non-ASCII characters from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#         new_words.append(new_word)
#     return new_words

# def to_lowercase(words):
#     """Convert all characters to lowercase from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = word.lower()
#         new_words.append(new_word)
#     return new_words

# def remove_punctuation(words):
#     """Remove punctuation from list of tokenized words"""
#     new_words = []
#     for word in words:
#         new_word = re.sub(r'[^\w\s]', '', word)
#         if new_word != '':
#             new_words.append(new_word)
#     return new_words

# def remove_stopwords(words):
#     """Remove stop words from list of tokenized words"""
#     new_words = []
#     for word in words:
#         if word not in stopwords.words('english'):
#             new_words.append(word)
#     return new_words

# def stem_words(words):
#     """Stem words in list of tokenized words"""
#     stemmer = LancasterStemmer()
#     stems = []
#     for word in words:
#         stem = stemmer.stem(word)
#         stems.append(stem)
#     return stems

# def lemmatize_verbs(words):
#     """Lemmatize verbs in list of tokenized words"""
#     lemmatizer = WordNetLemmatizer()
#     lemmas = []
#     for word in words:
#         lemma = lemmatizer.lemmatize(word, pos='v')
#         lemmas.append(lemma)
#     return lemmas

# def normalize(words): # preprocessing step
#     words = remove_non_ascii(words)
#     words = to_lowercase(words)
#     words = remove_punctuation(words)
# #     words = remove_stopwords(words)
#     return words

# would be faster to preprocess before tokenizing, I think

In [13]:
# Get corpus: 20 news groups

newsgroups_train = fetch_20newsgroups(subset='train')

# tokenize and preprocess

tokenizer = RegexpTokenizer(u'(?ui)\\b[a-z]{3,}\\b')

tokenized_corpus = [tokenizer.tokenize(document.lower()) for document in newsgroups_train.data]

# normalize_corpus = [normalize(document) for document in tokenized_corpus]

In [14]:
# get vocabulary

flattened = [word for document in tokenized_corpus for word in document]
vocabulary = set(flattened)
vocabulary = list(vocabulary)
len(vocabulary)

81072

In [ ]:
# # get vocabulary: with some preprocessing
# vectorizer = CountVectorizer(stop_words = 'english', 
#                              min_df=.01, max_df=0.9, 
#                              token_pattern = u'(?ui)\\b[a-z]{3,}\\b')

# count_vecs = vectorizer.fit_transform(newsgroups_train.data)

# # doc_term_matrix = count_vecs.toarray()
# # doc_term_matrix.shape # number of documents, number of words (in vocab)

# # note: vectorizer.get_feature_names() != vectorizer.vocabulary_

# vocabulary = vectorizer.get_feature_names()

In [19]:
# # vocabulary = []
# # for sentence in tokenized_corpus:
# #     for token in sentence:
# #         if token not in vocabulary:
# #             vocabulary.append(token)

word2idx = {w: idx + 1 for (idx , w) in enumerate(vocabulary)}
idx2word = {idx + 1: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

# idx_pairs = []
# # for each sentence
# for sentence in tokenized_corpus:
#     indices = [word2idx[word] for word in sentence]
#     # for each word, threated as center word
#     for center_word_pos in range(len(indices)):
#         # for each window position
#         for w in range(-window_size, window_size + 1):
#             context_word_pos = center_word_pos + w
#             # make soure not jump out sentence
#             if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
#                 continue
#             context_word_idx = indices[context_word_pos]
#             idx_pairs.append((indices[center_word_pos], context_word_idx))

# idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [22]:
indexed_corpus = []
# for each document
for document in tokenized_corpus:
    ragged_array = []
    # for each word
    for word in document:
        ragged_array.append(word2idx[word])
    indexed_corpus.append(ragged_array)

In [24]:
print(tokenized_corpus[0])
print(len(tokenized_corpus[0]))
print(indexed_corpus[0])
print(len(indexed_corpus[0]))

['from', 'lerxst', 'wam', 'umd', 'edu', 'where', 'thing', 'subject', 'what', 'car', 'this', 'nntp', 'posting', 'host', 'wam', 'umd', 'edu', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'anyone', 'out', 'there', 'could', 'enlighten', 'this', 'car', 'saw', 'the', 'other', 'day', 'was', 'door', 'sports', 'car', 'looked', 'from', 'the', 'late', 'early', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'the', 'body', 'this', 'all', 'know', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'where', 'this', 'car', 'made', 'history', 'whatever', 'info', 'you', 'have', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'brought', 'you', 'your', 'neighborhood', 'lerxst']
97
[78448, 76643, 8341, 61107, 11216, 44509, 6544, 59970, 71648, 41898, 46184, 52833, 33033, 70083, 8341, 61107, 11216, 67517, 32560, 

In [26]:
padded_corpus = [np.pad(document, (window,window), 'constant', constant_values=(0, 0)) 
                 for document in indexed_corpus]



In [35]:
test

test = padded_corpus[0]
contexts = []
for context in range(window, len(test) - window):
    center_word = test[context]
    context_words = np.concatenate((test[(context - window) : context], 
                                    test[context + 1 : context + window + 1]))
    contexts.append((center_word, context_words))
print(len(contexts))
print(len(indexed_corpus[0]))
contexts

array([    0,     0, 78448, 76643,  8341, 61107, 11216, 44509,  6544,
       59970, 71648, 41898, 46184, 52833, 33033, 70083,  8341, 61107,
       11216, 67517, 32560, 52327, 35572, 28150, 35811, 54779, 55247,
       55823, 53054, 45688, 14352, 72340, 46184, 41898, 52948, 13468,
       22816, 10371, 54779, 13931, 79462, 41898, 20817, 78448, 13468,
       14074, 10331, 54779, 24786, 10833, 13468, 12834, 26739, 66602,
       48261, 30054, 13468,  6310, 13749, 54779, 26931, 78448, 13468,
       35761, 13468, 42671, 46184, 64612, 56601, 55823, 25734, 46800,
        7313, 46382, 80707,  1513, 18684, 62633, 44509, 46184, 41898,
       45769, 80559, 79237, 45502, 19634, 45180, 46184, 48473, 26767,
       41898, 39163, 68034, 65412, 70832, 19634, 40506, 58495, 76643,
           0,     0])

In [38]:
test = padded_corpus[0]
contexts = []
for context in range(window, len(test) - window):
    center_word = test[context]
    context_words = np.concatenate((test[(context - window) : context], 
                                    test[context + 1 : context + window + 1]))
    contexts.append((center_word, context_words))
print(len(contexts))
print(len(indexed_corpus[0]))
contexts

97
97


[(78448, array([    0,     0, 76643,  8341])),
 (76643, array([    0, 78448,  8341, 61107])),
 (8341, array([78448, 76643, 61107, 11216])),
 (61107, array([76643,  8341, 11216, 44509])),
 (11216, array([ 8341, 61107, 44509,  6544])),
 (44509, array([61107, 11216,  6544, 59970])),
 (6544, array([11216, 44509, 59970, 71648])),
 (59970, array([44509,  6544, 71648, 41898])),
 (71648, array([ 6544, 59970, 41898, 46184])),
 (41898, array([59970, 71648, 46184, 52833])),
 (46184, array([71648, 41898, 52833, 33033])),
 (52833, array([41898, 46184, 33033, 70083])),
 (33033, array([46184, 52833, 70083,  8341])),
 (70083, array([52833, 33033,  8341, 61107])),
 (8341, array([33033, 70083, 61107, 11216])),
 (61107, array([70083,  8341, 11216, 67517])),
 (11216, array([ 8341, 61107, 67517, 32560])),
 (67517, array([61107, 11216, 32560, 52327])),
 (32560, array([11216, 67517, 52327, 35572])),
 (52327, array([67517, 32560, 35572, 28150])),
 (35572, array([32560, 52327, 28150, 35811])),
 (28150, array([

In [41]:
def document_contexts(document, window):
    contexts = []
    for context in range(window, len(document) - window):
        center_word = document[context]
        context_words = np.concatenate((document[(context - window) : context], 
                                        document[(context + 1) : (context + window + 1)]))
        contexts.append((center_word, context_words))
    return contexts

corpus_document_contexts = [document_contexts(document, window) for document in padded_corpus]



In [46]:
# we just care about contexts right now, so flatten the list
corpus_contexts = [pair for document_contexts in corpus_document_contexts for pair in document_contexts]

